In [32]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import fastparquet


In [33]:

NUM_CHANNELS = 3  # RGB


In [34]:
# Load the data
data = pd.concat([
    fastparquet.ParquetFile('./download/QCDToGGQQ_IMGjet_RH1all_jet0_run0_n36272.test.snappy.parquet').to_pandas(),
    fastparquet.ParquetFile('./download/QCDToGGQQ_IMGjet_RH1all_jet0_run1_n47540.test.snappy.parquet').to_pandas(),
    fastparquet.ParquetFile('./download/QCDToGGQQ_IMGjet_RH1all_jet0_run2_n55494.test.snappy.parquet').to_pandas()
])

# Shuffle the data
data = data.sample(frac=1).reset_index(drop=True)

# Split the data into training and testing sets
train_data = data.iloc[:90000]
test_data = data.iloc[90000:]


In [35]:
def preprocess_images(images):
    # Reshape the images
    images = images.reshape(NUM_CHANNELS)
    # Convert the images to floats between 0 and 1
    images = images.astype('float32') / 255.0
    return images


In [36]:
print(train_data.columns)

print(train_data.shape)

print(list(data.keys()))

Index(['X_jets', 'pt', 'm0', 'y'], dtype='object')
(90000, 4)
['X_jets', 'pt', 'm0', 'y']


In [37]:
# Preprocess the training and testing images
train_images = preprocess_images(train_data.drop(columns=['X_jets']).values)
test_images = preprocess_images(test_data.drop(columns=['X_jets']).values)

# Get the training and testing labels
train_labels = train_data['y'].values
test_labels = test_data['y'].values


ValueError: cannot reshape array of size 270000 into shape (3,)

In [29]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, NUM_CHANNELS)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(NUM_CLASSES, activation='softmax')
])


In [30]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [31]:
history = model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_split=0.1)


ValueError: Training data contains 0 samples, which is not sufficient to split it into a validation and training set as specified by `validation_split=0.1`. Either provide more data, or a different value for the `validation_split` argument.